In [26]:
import numpy as np
import requests
from requests_html import HTMLSession
from twilio.rest import Client
from bs4 import BeautifulSoup
from time import time


In [3]:
lux = 'https://www.on-site.com/web/online_app/choose_unit?goal=6&attr=x20&property_id=225322&lease_id=0&unit_id=0&required='
three_60 = 'https://www.on-site.com/web/online_app/choose_unit?goal=6&attr=x20&property_id=191247&lease_id=0&unit_id=0&required='

In [4]:
session = HTMLSession()
req = session.get(lux)
lux_paragraphs = req.html.find('p')
lux_paragraphs = lux_paragraphs[:-3]

In [5]:
session = HTMLSession()
req = session.get(three_60)
three_60_paragraphs = req.html.find('p')
three_60_paragraphs = three_60_paragraphs[:-3]

In [6]:
[p.text for p in lux_paragraphs]# if p.attrs['class'][0] == 'floor_plan_name']

['Lease Add-On', 'Room w/ shared bath, 500 Sq. Ft.']

In [7]:
[p.text for p in three_60_paragraphs]# if p.attrs['class'][0] == 'floor_plan_name']

['Lease Add-On',
 'Room w/ shared bath, 500 Sq. Ft.',
 'Compass 1A Studio',
 'Studio w/ bath, 662 Sq. Ft.',
 'Escape 1A 1 Bed/1 Ba',
 '1 bed, 1 bath, 874 Sq. Ft.',
 'Horizon 2B-G1 2 bed/2 ba',
 '2 bed, 2 bath, 1128 Sq. Ft.',
 'Panorama 2B 2 Bed/2 Ba',
 '2 bed, 2 bath, 1154 Sq. Ft.',
 'Infinity 2A 2 bed/2 ba',
 '2 bed, 2 bath, 1310 Sq. Ft.']

In [8]:
a = lux_paragraphs[0]

In [9]:
type(a.attrs['class'][0])

str

In [10]:
lux_dict = {}
current_plan = ""

for p in lux_paragraphs:
    # print(p.text)
    if (p.attrs['class'][0] == 'floor_plan_name') & (p.text not in lux_dict):
        # print("adding " + p.text + " to dict")
        lux_dict[p.text] = []
        current_plan = p.text
        # print(lux_dict)
    else:
        # print("adding " + p.text + " to " + current_plan)
        lux_dict[current_plan].append(p.text)
    # print(lux_dict)
print(lux_dict)
    

{'Lease Add-On': ['Room w/ shared bath, 500 Sq. Ft.']}


In [11]:
def make_dict(ps):
    current_plan = ""
    d = {}
    for p in ps:
        if (p.attrs['class'][0] == 'floor_plan_name') & (p.text not in d):
            d[p.text] = []
            current_plan = p.text
        else:
            d[current_plan].append(p.text)
    return d

In [12]:
three_60_d = make_dict(three_60_paragraphs)
lux_d = make_dict(lux_paragraphs)

In [13]:
print(lux_d.keys())

dict_keys(['Lease Add-On'])


In [21]:
from twilio.rest import Client 
 
account_sid = 'ACb89be4618d01504371ee5a71581e0a4d' 
auth_token = '257e09038f86713f5d78f6e025a0ebdb' 
client = Client(account_sid, auth_token) 
 
message = client.messages.create(  
                              messaging_service_sid='MG9b262dfc07ca251f557ea5206000f0cc', 
                              body='update:' + str(lux_d.keys()),      
                              to='+16502914624') 
 
print(message.sid)

TwilioException: Credentials are required to create a TwilioClient

In [16]:
def get_difference(d_before, d_current):
    message = ""
    if (len(d_before)) > len(d_current):
        taken = "\n".join(d_before.keys() - d_current.keys())
        message = "Listing removed update: " + taken
    else:
        added = "\n".join(d_current.keys() - d_before.keys())
        message = "New listing update: " + added
    return message

In [19]:
get_difference(lux_d, three_60_d)

'New listing update: Panorama 2B 2 Bed/2 Ba\nInfinity 2A 2 bed/2 ba\nEscape 1A 1 Bed/1 Ba\nHorizon 2B-G1 2 bed/2 ba\nCompass 1A Studio'

In [20]:
"\n".join(lux_d.keys())

'Lease Add-On'

In [22]:
def get_html(url):
    session = HTMLSession()
    req = session.get(url)
    paragraphs = req.html.find('p')
    paragraphs = paragraphs[:-3]

    return paragraphs

def make_dict(ps):
    current_plan = ""
    d = {}
    for p in ps:
        if (p.attrs['class'][0] == 'floor_plan_name') & (p.text not in d):
            d[p.text] = []
            current_plan = p.text
        else:
            d[current_plan].append(p.text)
    return d

def get_difference(d_before, d_current):
    message = ""
    if (len(d_before)) > len(d_current):
        taken = ", ".join(d_before.keys() - d_current.keys())
        message = "Listing removed update: " + taken
    else:
        added = ", ".join(d_current.keys() - d_before.keys())
        message = "New listing update: " + added

    return message

def send_sms(message):
    account_sid = 'ACb89be4618d01504371ee5a71581e0a4d' 
    auth_token = '257e09038f86713f5d78f6e025a0ebdb' 
    client = Client(account_sid, auth_token) 
    
    message = client.messages.create(messaging_service_sid='MG9b262dfc07ca251f557ea5206000f0cc', body=message, to='+16502914624') 

In [23]:
minute_before_lux_d = make_dict(get_html(three_60))
minute_before_three_60_d = make_dict(get_html(lux))

In [34]:
lux = 'https://www.on-site.com/web/online_app/choose_unit?goal=6&attr=x20&property_id=225322&lease_id=0&unit_id=0&required='
three_60 = 'https://www.on-site.com/web/online_app/choose_unit?goal=6&attr=x20&property_id=191247&lease_id=0&unit_id=0&required='

minute_before_lux = {}
minute_before_three_60 = {}

def get_html(url):
    session = HTMLSession()
    req = session.get(url)
    paragraphs = req.html.find('p')
    paragraphs = paragraphs[:-3]

    return paragraphs

def make_dict(ps):
    current_plan = ""
    d = {}
    for p in ps:
        if (p.attrs['class'][0] == 'floor_plan_name') & (p.text not in d):
            d[p.text] = []
            current_plan = p.text
        else:
            d[current_plan].append(p.text)
    return d

def get_difference(d_before, d_current):
    message = ""
    if (len(d_before)) > len(d_current):
        taken = ", ".join(d_before.keys() - d_current.keys())
        message = "Listing removed update: " + taken
    else:
        added = ", ".join(d_current.keys() - d_before.keys())
        message = "New listing update: " + added

    return message

def send_sms(message):
    account_sid = 'ACb89be4618d01504371ee5a71581e0a4d' 
    auth_token = '257e09038f86713f5d78f6e025a0ebdb' 
    client = Client(account_sid, auth_token) 
    
    # sending to so
    message = client.messages.create(messaging_service_sid='MG9b262dfc07ca251f557ea5206000f0cc', body=message, to='+16502914624') 
    # sending to zack
    # message = client.messages.create(messaging_service_sid='MG9b262dfc07ca251f557ea5206000f0cc', body=message, to='+17605045501') 
    # sending to peter
    # message = client.messages.create(messaging_service_sid='MG9b262dfc07ca251f557ea5206000f0cc', body=message, to='+16193175276') 
    


In [39]:
three_60_d = make_dict(get_html(three_60))
lux_d = make_dict(get_html(lux))

if len(minute_before_three_60_d) != len(lux_d):
    print("sending message")
    message = get_difference("Lux UTC", minute_before_three_60_d, lux_d)
    minute_before_lux_d = lux_d
    send_sms(message)

if len(minute_before_three_60_d) != len(three_60_d):
    message = get_difference("360 Lux", minute_before_three_60_d, three_60_d)
    minute_before_three_60_d = three_60_d
    send_sms(message)

sending message


In [28]:
import time

In [37]:
def get_difference(name, d_before, d_current):
    message = ""
    if (len(d_before)) > len(d_current):
        taken = ", ".join(d_before.keys() - d_current.keys())
        message = "[" + name + "] " + "Listing removed update: " + taken
    else:
        added = ", ".join(d_current.keys() - d_before.keys())
        message = "[" + name + "] " + "New listing update: " + added

    return message

In [40]:
time.cu

<module 'time' (built-in)>

In [45]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)	

now = 2022-04-22 14:20:39.182014
date and time = 22/04/2022 14:20:39
